---
title: Data ingestion
format: html
toc: true
lang: en
jupyter: python3
ipynb-shell-interactivity: all
execute:
  echo: false
date: 2025-08-09 08:02 +0200
categories: ["datascience", "fundamentals"]
tags: ["notes", "dataingestion"]

comments:
  giscus:
    repo: jeev20/jeev20.github.io
---


## Getting the data
The location of the data depends on ones use-case. Some have it locally, others on the cloud in a storage bucket or database. There is always a way to get your data to your development environment. The way we get it will differ. 

In this notes, I use the `kagglehub` module to get a time series dataset. 

```{.python}
import kagglehub
# Download latest version
path = kagglehub.dataset_download("garystafford/environmental-sensor-data-132k")
print("Path to dataset files:", path)
```

## Decrease data size
One needs to evaluate the size of the dataset and the resources available to process the data. One way og limiting the size of the data is to use effecient file formats.

Data formatted as Comma Seperated Value (CSV) is everywhere, but it is not the most lighweight or fast format when it comes read/write from disks. So it is wise to convert large CSV files to formats which are faster and take lesser space on disk/memory. One such format is [`parquet`](https://www.databricks.com/glossary/what-is-parquet). 

We can always decrease the data size to make ingestion easier. So we use polars to convert it to paraquet format. 

In [ ]:
#| echo: true
%%time
import polars as pl

input_data_path = f"../data/iot/iot_telemetry_data.parquet"
df = pl.scan_csv("../data/iot/iot_telemetry_data.csv")
df.sink_parquet(input_data_path)  # Saves the lazyframe as a parquet file

For small size datasets pandas and polars will do fine. As the dataset size increases, we need to look for effecient ways to read and prosess our data. In python, DuckDb and Pyspark are the best performing ETL libraries for large datasets. 

That said, output from both DuckDb and Pyspark are not directly compatible with visualization libraries or other third party modules, for example, `pandas-profiling`. 

So a hybrid approach is required, where the transformations are made using DuckDb or Pyspark, but the output is later converted to either polars or pandas dataframes. This allows us to efficiently perform ETL operations, but still be compatible with visualization libraries via polars or pandas formats. 



## Using pandas

In [ ]:
#| echo: true
%%time
import pandas as pd

pd_df =  pd.read_parquet(input_data_path)
pd_df.head(2)

## Using Polars

In [ ]:
#| echo: true
%%time
import polars as pl

pl_df =  pl.scan_parquet(input_data_path)
pl_df.head(2).collect()

## Using Duckdb

In [ ]:
#| echo: true
%%time
import duckdb 

result = duckdb.sql(f"SELECT * FROM '{input_data_path}'")
result.show()

## Using pyspark

In [ ]:
#| echo: true
#| warning: false
%%time
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("readParquet").getOrCreate()
df = spark.read.parquet(input_data_path)
df.show()